In [1]:
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math


In [2]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')

# set up concat directories
targetFolder = Path(home, 'HP Inc','GPSTW SOP - 2021 日新','Project team','Upload folder ( for buyer update )')
FD_folder = Path(targetFolder, "FD_temp","*.xlsx")
shortage_folder = Path(targetFolder ,"shortage_temp","*.xlsx")
PNbaseDetail_folder = Path(targetFolder ,"PNbasedDetail_temp","*.xlsx")

In [3]:
# concat FD
FD_files = []
for FD_file in glob.glob(str(Path(FD_folder))):
    FD = pd.read_excel(FD_file)
    FD_files.append(FD)
FD_all = pd.concat(FD_files)


In [4]:
# concat shortage
shortage_files = []
for shortage_file in glob.glob(str(Path(shortage_folder))):
    shortage = pd.read_excel(shortage_file)
    shortage_files.append(shortage)
Shortage_all = pd.concat(shortage_files)

In [5]:
# concat PNbaseDetail
PNbasedDetail_files = []
for PNbasedDetail_file in glob.glob(str(Path(PNbaseDetail_folder))):
    PNbasedDetail = pd.read_excel(PNbasedDetail_file)
    PNbasedDetail_files.append(PNbasedDetail)
PNbaseDetail_all = pd.concat(PNbasedDetail_files)

In [6]:
Shortage_all = Shortage_all.sort_values(['Platform','FV'], ascending=[True,True])
Shortage_all

,ODM,Item,Commodity,FV,Platform,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date
1,CQQCI,2.0,BATTERY,PP03043XL PL,Able 20C2/Able 21C2/Dezik 20C2/Dezik 21C2/Trix...,0,0,0,1888,0,0,2022-10-19,NaT
1,CQQCI,2.0,BATTERY,PP03043XL PL,Able 20C2/Able 21C2/Dezik 20C2/Dezik 21C2/Trix...,12552,0,0,12552,0,0,2022-10-20,NaT
3,CQIEC,4.0,CONNECTOR,DCJK 9p TH H7.45 C4.5,Adicora/Adicora R,996603,0,299000,1295603,0,0,2022-09-19,2022-11-07 00:00:00
3,CQIEC,4.0,CONNECTOR,DCJK 9p TH H7.45 C4.5,Adicora/Adicora R,931831,0,185000,1116831,0,0,2022-09-20,2022-11-07 00:00:00
4,CQIEC,4.0,CONNECTOR,DCJK 9p TH H7.45 C4.5,Adicora/Adicora R,894355,0,0,894355,0,0,2022-09-21,2022-10-24 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,CQQCI,2.0,SSS,SSD 512GB+32GB 2280 PCIe-3x2x2 NVMe,Wikus 21C1/Reeds/Rabon/Raritan,10,0,0,10,0,10,2022-10-12,2050-01-01 00:00:00
0,CEI,BATT84,BATTERY,WF04066XL PL,Woodford 22C1,2677,0,0,2677,0,0,2022-10-24,2022-10-24 00:00:00
1,CQWIS,2.0,EMMC,eMMC 32GB v5.0,YELLOWFIN,0,0,0,0,0,0,2022-10-20,NaN
0,CQWIS,1.0,EMMC,eMMC 64GB v5.0,YELLOWFIN,0,0,0,0,0,0,2022-10-20,NaN


In [7]:
FD_all = FD_all.sort_values(['Platform','FV'], ascending=[True,True])
FD_all

,ODM,Item,Commodity,FV,Platform,Supplier,HP PN,ReportDate,FDdate,FDQty
0,CQIEC,4.0,CONNECTOR,DCJK 9p TH H7.45 C4.5,Adicora/Adicora R,FIT,L56957-F75,2022-09-19,2022-09-19,180840
9,CQIEC,4.0,CONNECTOR,DCJK 9p TH H7.45 C4.5,Adicora/Adicora R,SINGATRON,L56957-S75,2022-09-19,2022-09-22,42500
10,CQIEC,4.0,CONNECTOR,DCJK 9p TH H7.45 C4.5,Adicora/Adicora R,SUYIN-CORP,L56957-Y75,2022-09-19,2022-09-22,70000
20,CQIEC,4.0,CONNECTOR,DCJK 9p TH H7.45 C4.5,Adicora/Adicora R,SINGATRON,L56957-S75,2022-09-19,2022-09-29,80000
24,CQIEC,4.0,CONNECTOR,DCJK 9p TH H7.45 C4.5,Adicora/Adicora R,FIT,L56957-F75,2022-09-19,2022-09-30,101640
...,...,...,...,...,...,...,...,...,...,...
6,WHFXN,14,CPU,AMD CEZANNE RYZEN3 5300G AM4,WatsonA/MarvelA,NaN,M46322-A03,2022-09-19,2023-01-23,180
7,WHFXN,14,CPU,AMD CEZANNE RYZEN3 5300G AM4,WatsonA/MarvelA,NaN,M46322-A03,2022-09-19,2023-01-30,240
8,WHFXN,14,CPU,AMD CEZANNE RYZEN3 5300G AM4,WatsonA/MarvelA,NaN,M46322-A03,2022-09-19,2023-02-06,360
9,WHFXN,14,CPU,AMD CEZANNE RYZEN3 5300G AM4,WatsonA/MarvelA,NaN,M46322-A03,2022-09-19,2023-02-13,180


In [8]:
PNbaseDetail_all = PNbaseDetail_all.sort_values(['GPS Remark','ODM use column1','ODM use column2','ODM use column3','ODM use column4','ODM use column5'], ascending=[True,True,True,True,True,True])
PNbaseDetail_all.head(10)

,Item,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate
6,7.0,L93741-4A0,468pcs on hold for sorting,13,468,0,NaN,NaN,NaN,NaN,NaN,2022-10-12
6,7.0,L93741-4A0,546pcs on hold for sorting,0,546,0,NaN,NaN,NaN,NaN,NaN,2022-10-11
0,BATT70,L08934-2B2,"EOL platform, ask region to cancel orders",216,0,0,NaN,NaN,NaN,NaN,NaN,2022-09-22
3,6.0,M09631-001,EOL without inventory,0,0,0,NaN,NaN,NaN,NaN,NaN,2022-09-23
2,3.0,L50352-001,"L50352-001 is EOL, please notify CIS team to E...",0,0,0,CIS order use L50352-001/M10394-003 only,NaN,NaN,NaN,NaN,2022-09-26
1,2.0,L50352-001,"L50352-001 is EOL, please notify CIS team to E...",0,0,0,NaN,NaN,NaN,NaN,NaN,2022-09-26
1,3.0,L70939-001,L70939-001 is EOL. Need CIS team to do EM to M...,0,0,0,"CIS order use L70939-001 only, M54701-002 need...",NaN,NaN,NaN,NaN,2022-09-23
0,1.0,L70939-001,L70939-001 is EOL. Need CIS team to do EM to M...,0,0,0,"CIS order use L70939-001 only, M54701-002 need...",NaN,NaN,NaN,NaN,2022-09-26
0,1,M90877-301,"Special Sku for India Factory, not VMI B/S p...",0,0,0,NaN,NaN,NaN,NaN,NaN,2022-10-20
1,10.0,L80992-Y32,NaN,0,0,0,/0GAH/0GE/0GJ,Hornbill/Ovenbird/Hitchcock,NaN,NaN,NaN,2022-09-19


In [9]:
# apache airflow to upload SQL ( currently to desktop )
FD_all.to_excel(Path(home, 'Desktop', 'FD_all.xlsx'), index=False)
Shortage_all.to_excel(Path(home, 'Desktop', 'Shortage_all.xlsx'), index=False)
PNbaseDetail_all.to_excel(Path(home, 'Desktop', 'PNbaseDetail_all.xlsx'), index=False)